In [ ]:
#If you’re happy with the dataset, then load it
from datasets import load_dataset
from datasets import get_dataset_split_names
from datasets import load_dataset
from datasets import get_dataset_config_names
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
# Load datasets
tifu_short_raw = load_dataset("reddit_tifu", "short", split="train")
tifu_long_raw = load_dataset("reddit_tifu", "long", split = "train")


In [ ]:
# Establish model name:

model_name = "bert-base-uncased"

In [ ]:
# Add decimal labels to upvote ratio
def add_columns(example):
    example["upvote_ratio"] = round(example["upvote_ratio"], 1)
    example["labels"] = int(example["upvote_ratio"] * 10)
    return example

tifu_short = tifu_short_raw.map(add_columns)

tifu_long = tifu_long_raw.map(add_columns)

In [ ]:
# Split into train, validation and test. MAKE THIS A FUNCTION
dict1 = tifu_short.train_test_split(test_size=0.1)

train_and_val = dict1["train"]
test_short = dict1["test"]

dict2 = train_and_val.train_test_split(test_size=0.2)

train_short = dict2["train"]
val_short = dict2["test"]


In [ ]:
print(tifu_short)
print(train_short)
print(val_short)
print(test_short)

In [ ]:
#print(len(train_short[0]["documents"]))

In [ ]:
# Tokenize # MAKE THIS AND OTHER PREPROCESSING A FUNCTION
tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenization(example):

    return tokenizer(example["documents"], padding = True, truncation= True, return_tensors = "pt")




train_input = train_short.map(tokenization, batched = True)
val_input = val_short.map(tokenization, batched = True)
test_input = test_short.map(tokenization, batched = True)

In [ ]:

# def tokenization(example):

#     return tokenizer(example["documents"])

# short_diagnostics = train_short.map(tokenization, batched = True)

In [ ]:
# len(short_diagnostics["input_ids"])

#a = short_diagnostics[1]
#len(a["input_ids"])

In [ ]:
# def add_length(example):
#     example["input_length"] = len(example["input_ids"])
#     return example

# short_diagnostics = short_diagnostics.map(add_length)

In [ ]:
# from matplotlib import pyplot as plt

# plt.plot(short_diagnostics["input_length"][2000:10000])
# min(short_diagnostics["input_length"]) # 3
# sum(short_diagnostics["input_length"]) / len(short_diagnostics["input_length"])

# k = 0
# for i in short_diagnostics["input_length"]:
#     if i > 1000:
#         k += 1
# print(k)


In [ ]:
# short_diagnostics["input_length"].index(max(short_diagnostics["input_length"]))

In [ ]:
# short_diagnostics[38395]["input_length"]

In [ ]:
train_input.save_to_disk("preprocessed/train_short")
val_input.save_to_disk("preprocessed/val_short")
test_input.save_to_disk("preprocessed/test_short")

In [ ]:
from datasets import load_from_disk

train_cl_short = load_from_disk("preprocessed/train_short")
val_cl_short = load_from_disk("preprocessed/val_short")
test_cl_short = load_from_disk("preprocessed/test_short")


In [ ]:
#print(train_cl_short)
#print(val_cl_short)
print(test_cl_short[0]["labels"])

In [126]:
# Let's train a model!

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_cl_short, shuffle=True, batch_size=32)

val_dataloader = DataLoader(val_cl_short, batch_size=32)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10)


from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3

num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(

    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps

)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [127]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):

    for batch in train_dataloader:

        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)

        loss = outputs.loss

        loss.backward()

        optimizer.step()

        lr_scheduler.step()

        optimizer.zero_grad()

        progress_bar.update(1)

  0%|          | 0/5385 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'to'

In [125]:
import evaluate

metric = evaluate.load("accuracy")

model.eval()

for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

torch.utils.data.dataloader.DataLoader

In [ ]:
nosave = 0
# Train model


print("[INFO:] Training classifier...")


for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# Prepare validation dataset in a single batch
# For every epoch
for epoch in range(epochs):
    ## for each batch
    for batch in batches:
            ## prepare batch
        token_idx, tags = prepare_batch(batch[0], batch[1], vocab)

        ## train on one batch
            # FORWARD PASS
        X = token_idx
        y = model(X)
            # Calculate loss
        loss = model.loss_fn(outputs=y, labels=tags)
            # backpropagation
        loss.backward()
            # take step, reset
        optimizer.step()
        optimizer.zero_grad()
        # Validate model
    val_X = val_idx
    val_y = model(val_X)
        # Calculate validation loss
    val_loss = model.loss_fn(outputs=val_y, labels=val_tags)
        # If best_val_loss exists, see if it needs updating.
    try:
        if best_val_loss > val_loss.item():
            best_val_loss = val_loss.item()
                # save the model if it is the best so far
            torch.save(model, f"models/{model_name}")
            nosave = 0
        else:
                # Increment nosave
            nosave += 1
        # Otherwise, create it and save the model.
    except NameError:
        best_val_loss = val_loss.item()
        torch.save(model, f"models/{model_name}")
        # stop the training if patience is used up
    if nosave > 5:
        print("Patience is up!")
        break
        # some print to see that it is running
        #if (epoch + 1) % 10 == 0: # Comment or uncomment depending on how many epochs you want printed.
    print(f"epoch: {epoch+1}, loss = {loss.item():.4f}, best validation loss = {best_val_loss:.4f}")

print("[INFO:] Finished traning!")


PREPROCESSING DECISIONS:

1. Labels at every tenth percentile.

2. Max input sequence length (What do we do with those that exceed it? Do we truncate and hope for the best?)

Test the first: Truncation time!

3. Special characters, URLs, usernames, subreddit-names (Maybe the latter two are less relevant?).

In [ ]:
# Giant code dump from the Huggingface documentation.
# Don't even try to run this!

from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)


encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")

print(encoding)
{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

pt_batch = tokenizer(

    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],

    padding=True,

    truncation=True,

    max_length=512,

    return_tensors="pt",

)


from transformers import AutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

pt_outputs = pt_model(**pt_batch)

from torch import nn

pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)

print(pt_predictions)
tensor([[0.0021, 0.0018, 0.0115, 0.2121, 0.7725],
        [0.2084, 0.1826, 0.1969, 0.1755, 0.2365]], grad_fn=<SoftmaxBackward0>)


pt_save_directory = "./pt_save_pretrained"

tokenizer.save_pretrained(pt_save_directory)

pt_model.save_pretrained(pt_save_directory)

pt_model = AutoModelForSequenceClassification.from_pretrained("./pt_save_pretrained")


from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)

pt_model = AutoModelForSequenceClassification.from_pretrained(tf_save_directory, from_tf=True)


from transformers import AutoConfig

my_config = AutoConfig.from_pretrained("distilbert-base-uncased", n_heads=12)

from transformers import AutoModel

my_model = AutoModel.from_config(my_config)

# Huggingface PyTorch Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(

    output_dir="path/to/save/folder/",

    learning_rate=2e-5,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=8,

    num_train_epochs=2,

)

# Preprocessing Class
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocessed train and test data
train_dataset = dataset["train"]  # doctest: +SKIP

eval_dataset = dataset["eval"]  # doctest: +SKIP

# DataCollator to create examples
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# Trainer!
from transformers import Trainer

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=dataset["train"],

    eval_dataset=dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

)  
# trainer.train()

